In [1]:
!pip3 install pyro-ppl 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 718 kB 5.4 MB/s 


In [2]:
import os, torch, pyro
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pyro.distributions as dist
from torch import nn
from functools import partial
from pyro.nn import PyroModule, PyroSample
from pyro.infer import SVI, Trace_ELBO
from pyro.infer.autoguide import AutoDiagonalNormal

In [10]:
pyro.set_rng_seed(1)
pyro.enable_validation(True)
%matplotlib inline
plt.style.use('default')

def get_data():
    DATA_URL = "https://d2hg8soec8ck9v.cloudfront.net/datasets/rugged_data.csv"
    data = pd.read_csv(DATA_URL, encoding="ISO-8859-1")
    df = data[["cont_africa", "rugged", "rgdppc_2000"]]
    df = df[np.isfinite(df.rgdppc_2000)]
    df["rgdppc_2000"] = np.log(df["rgdppc_2000"])
    df["cont_africa_x_rugged"] = df["cont_africa"] * df["rugged"]
    data = torch.tensor(df[["cont_africa", "rugged", "cont_africa_x_rugged", "rgdppc_2000"]].values,
                            dtype=torch.float)
    x_data, y_data = data[:, :-1], data[:, -1]
    return x_data, y_data

x_data, y_data = get_data()

num_iterations =1000

class BayesianRegression(PyroModule):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear = PyroModule[nn.Linear](in_features, out_features)
        self.linear.weight = PyroSample(dist.Normal(0., 1.).expand([out_features, in_features]).to_event(2))
        self.linear.bias = PyroSample(dist.Normal(0., 10.).expand([out_features]).to_event(1))

    def forward(self, x, y=None):
        sigma = pyro.sample("sigma", dist.Uniform(0., 10.))
        mean = self.linear(x).squeeze(-1)
        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", dist.Normal(mean, sigma), obs=y)
        return mean

model = BayesianRegression(3, 1)
guide = AutoDiagonalNormal(model)
adam = pyro.optim.Adam({"lr": 0.03})
svi = SVI(model, guide, adam, loss=Trace_ELBO())

pyro.clear_param_store()
for j in range(num_iterations):
    loss = svi.step(x_data, y_data)
    if j % 200 == 0:
        print("[iteration %04d] loss: %.4f" % (j + 1, loss / len(x_data)))

guide.requires_grad_(False)

for name, value in pyro.get_param_store().items():
    print(name, pyro.param(name))

AutoDiagonalNormal()
[iteration 0001] loss: 4.6074
[iteration 0201] loss: 2.5523
[iteration 0401] loss: 1.4601
[iteration 0601] loss: 1.4726
[iteration 0801] loss: 1.4677
AutoDiagonalNormal.loc Parameter containing:
tensor([-2.2916, -1.8635, -0.1926,  0.3306,  9.1682])
AutoDiagonalNormal.scale tensor([0.0559, 0.1429, 0.0459, 0.0846, 0.0633])


In [16]:
def data_generate():
  rand = torch.randn(100)
  x = rand
  y = rand * 4 + 2.
  return x, y

class RegressionModel(nn.Module):
  def __init__(self, p):
    super(RegressionModel, self).__init__()
    self.linear = PyroModule[nn.Linear](1, 1)
    self.linear.weight = PyroSample(dist.Normal(0., 1.).expand([1, 1]).to_event(2))
    self.linear.bias = PyroSample(dist.Normal(0., 10.).expand([1]).to_event(1))
  
  def forward(self, x, y=None):
    return self.linear(x)

x_data, y_data = data_generate()
x_data = x_data.reshape(100,1)
y_data = y_data.reshape(100,1)

num_iterations = 100
p = 1

model = RegressionModel(1)
guide = AutoDiagonalNormal(model)
adam = pyro.optim.Adam({"lr": 0.03})
svi = SVI(model, guide, adam, loss=Trace_ELBO())

pyro.clear_param_store()
for j in range(num_iterations):
    loss = svi.step(x_data, y_data)
    if j % 20 == 0:
        print("[iteration %04d] loss: %.4f" % (j + 1, loss / len(x_data)))

guide.requires_grad_(False)

for name, value in pyro.get_param_store().items():
    print(name, pyro.param(name))


[iteration 0001] loss: 0.0684
[iteration 0021] loss: 0.0566
[iteration 0041] loss: 0.0426
[iteration 0061] loss: 0.0275
[iteration 0081] loss: 0.0291
AutoDiagonalNormal.loc Parameter containing:
tensor([0.1420, 0.1224])
AutoDiagonalNormal.scale tensor([0.8826, 1.0108])


In [17]:
from collections import defaultdict
from pyro import poutine
from pyro.poutine.util import prune_subsample_sites
import warnings


class Predict(torch.nn.Module):
    def __init__(self, model, guide):
        super().__init__()
        self.model = model
        self.guide = guide

    def forward(self, *args, **kwargs):
        samples = {}
        guide_trace = poutine.trace(self.guide).get_trace(*args, **kwargs)
        model_trace = poutine.trace(poutine.replay(self.model, guide_trace)).get_trace(*args, **kwargs)
        for site in prune_subsample_sites(model_trace).stochastic_nodes:
            samples[site] = model_trace.nodes[site]['value']
        return tuple(v for _, v in sorted(samples.items()))

predict_fn = Predict(model, guide)
predict_module = torch.jit.trace_module(predict_fn, {"forward": (x_data,)}, check_trace=False)

In [21]:
from pyro.infer import Predictive

predictive = Predictive(model, guide=guide, num_samples=80,
                        return_sites=("linear.weight", "obs", "_RETURN"))
samples = predictive(x_data)